<a href="https://colab.research.google.com/github/nncliff/qwen-32B/blob/main/chapter-1/lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoRA (Low-Rank Adaptation) Implementation

This notebook implements LoRA (Low-Rank Adaptation) for a custom Transformer block (`QWQBlockWithLoRA`) on a dummy LegalQA dataset.

LoRA allows us to fine-tune large models by freezing the pre-trained model weights and injecting trainable rank decomposition matrices into each layer of the Transformer architecture.


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random

# Set seed for reproducibility
random.seed(42)
torch.manual_seed(42)


In [20]:
class LegalQADataset(Dataset):
    def __init__(self, num_samples=500):
        self.data = []

        for _ in range(num_samples):
            label = random.randint(0, 1) # Binary classification
            q_vec = torch.randn(64) + label * 0.5  # Simple pattern based on label
            a_vec = torch.randn(64) + label * 0.5 # Simple pattern based on label
            self.data.append((q_vec, a_vec, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        q, a, label = self.data[idx]
        return q, a, torch.tensor(label)


### Understanding `F.linear` and Matrix Shapes

**The Short Answer:**
`F.linear(input, weight)` **automatically transposes** the `weight` matrix.
It computes $y = x W^T$.

**The Detailed Explanation:**
1.  **Standard Math**: We usually write $y = xW$, where $x$ is $(1 \times D_{in})$ and $W$ is $(D_{in} \times D_{out})$.
2.  **PyTorch Convention**: PyTorch stores Linear layer weights as $(D_{out} \times D_{in})$ to optimize memory layout.
3.  **The Operation**: Because the weight is stored "transposed" relative to the math notation, `F.linear` performs the transpose for you during the forward pass.

**In the code below:**
*   `x` shape: `(batch_size, input_dim)`
*   `self.lora_A` shape: `(rank, input_dim)`

When calling `F.linear(x, self.lora_A)`:
*   It computes: $x \cdot \text{lora\_A}^T$
*   Shapes: `(batch, input_dim)` $\cdot$ `(input_dim, rank)`
*   Result: `(batch, rank)`

In [21]:
class LoRAModule(nn.Module):
    def __init__(self, input_dim, out_dim, rank=4, alpha=16):
        super(LoRAModule, self).__init__()
        self.input_dim = input_dim
        self.out_dim = out_dim
        self.rank = rank
        self.alpha = alpha
        self.scaling = self.alpha / self.rank

        # Original weight matrix (frozen during training)
        # Shape: (out_dim, input_dim)
        self.weight = nn.Parameter(torch.randn(out_dim, input_dim), requires_grad=False)

        # Trainable low-rank adaptation matrices
        # lora_A: (rank, input_dim)
        # lora_B: (out_dim, rank)
        self.lora_A = nn.Parameter(torch.randn(rank, input_dim) * 0.01)
        self.lora_B = nn.Parameter(torch.randn(out_dim, rank) * 0.01)

    def forward(self, x):
        # x: (batch_size, input_dim)

        # Original path: x @ weight.T
        # (batch_size, input_dim) @ (input_dim, out_dim) -> (batch_size, out_dim)
        original_output = F.linear(x, self.weight)

        # LoRA path: x @ lora_A.T @ lora_B.T
        # 1. x @ lora_A.T
        # (batch_size, input_dim) @ (input_dim, rank) -> (batch_size, rank)
        lora_output = F.linear(x, self.lora_A)

        # 2. result @ lora_B.T
        # (batch_size, rank) @ (rank, out_dim) -> (batch_size, out_dim)
        lora_output = F.linear(lora_output, self.lora_B) * self.scaling

        return original_output + lora_output


In [22]:
class QWQBlockWithLoRA(nn.Module):
    def __init__(self, input_dim=64):
        super(QWQBlockWithLoRA, self).__init__()
        self.q_proj = LoRAModule(input_dim, input_dim)
        self.k_proj = LoRAModule(input_dim, input_dim)
        self.v_proj = LoRAModule(input_dim, input_dim)
        self.out_proj = LoRAModule(input_dim, input_dim)
        self.ffn = nn.Sequential(
            LoRAModule(input_dim, input_dim * 2),
            nn.ReLU(),
            LoRAModule(input_dim * 2, input_dim)
        )

        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)

    def forward(self, q, k, v):
        # Self-attention
        q_proj = self.q_proj(q)
        k_proj = self.k_proj(k)
        v_proj = self.v_proj(v)

        # Scaled Dot-Product Attention
        attn_scores = torch.matmul(q_proj, k_proj.T) / (q_proj.size(-1) ** 0.5)
        attn_probs = F.softmax(attn_scores, dim=-1)
        attn_output = torch.matmul(attn_probs, v_proj)
        attn_output = self.out_proj(attn_output) # Optional output projection

        # Add & Norm
        x = self.norm1(q + attn_output)
        x = self.norm2(self.ffn(x) + x)

        return x


### Q&A Classification Strategy: Dual Encoder

In this `LegalQAClassifierWithLoRA`, we use a simple **Dual Encoder** architecture:

1.  **Independent Encoding**:
    *   The Question (`q`) and Answer (`a`) are processed separately by the *same* encoder (`self.encoder`).
    *   `self.encoder(q, q, q)` applies self-attention to the question vector to generate a context-aware representation.
    *   This results in two dense vectors: `q_encoded` and `a_encoded`.

2.  **Interaction (Element-wise Multiplication)**:
    *   `combined = q_encoded * a_encoded`
    *   We combine the two representations using element-wise multiplication (Hadamard product). This operation emphasizes features that are prominent in *both* the question and the answer, acting as a similarity measure.

3.  **Classification**:
    *   The combined vector is passed to a linear classifier to predict the relationship (e.g., relevant vs. irrelevant).

This approach is efficient and effective for simple matching tasks, as it forces the model to learn a shared embedding space where matching questions and answers have similar features.

In [23]:
class LegalQAClassifierWithLoRA(nn.Module):
    def __init__(self):
        super(LegalQAClassifierWithLoRA, self).__init__()
        self.encoder = QWQBlockWithLoRA()
        self.classifier = nn.Linear(64, 2)  # Binary classification

    def forward(self, q, a):
        # In this dummy example, we use the same input for q, k, v
        q_encoded = self.encoder(q, q, q)
        a_encoded = self.encoder(a, a, a)
        combined = q_encoded * a_encoded # Element-wise multiplication
        return self.classifier(combined)


In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = LegalQAClassifierWithLoRA().to(device)

# Only train parameters that require gradients (LoRA parameters)
# The original weights are frozen (requires_grad=False)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
criterion = nn.CrossEntropyLoss()

loader = DataLoader(LegalQADataset(), batch_size=32, shuffle=True)

model.train()
for epoch in range(30):
    total_loss = 0
    correct = 0
    for batch in loader:
        q_batch, a_batch, labels = batch
        q_batch, a_batch, labels = q_batch.to(device), a_batch.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(q_batch, a_batch)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()

    acc = correct / len(loader.dataset)
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}, Accuracy: {acc:.4f}")


Using device: cpu
Epoch 1, Loss: 0.7094, Accuracy: 0.5060
Epoch 2, Loss: 0.7080, Accuracy: 0.5040
Epoch 3, Loss: 0.7082, Accuracy: 0.5040
Epoch 4, Loss: 0.6803, Accuracy: 0.5720
Epoch 5, Loss: 0.6752, Accuracy: 0.5760
Epoch 6, Loss: 0.6658, Accuracy: 0.5680
Epoch 7, Loss: 0.6465, Accuracy: 0.6180
Epoch 8, Loss: 0.6205, Accuracy: 0.6760
Epoch 9, Loss: 0.5986, Accuracy: 0.6700
Epoch 10, Loss: 0.5961, Accuracy: 0.6860
Epoch 11, Loss: 0.5835, Accuracy: 0.7000
Epoch 12, Loss: 0.5314, Accuracy: 0.7400
Epoch 13, Loss: 0.5291, Accuracy: 0.7200
Epoch 14, Loss: 0.5226, Accuracy: 0.7500
Epoch 15, Loss: 0.5526, Accuracy: 0.7380
Epoch 16, Loss: 0.4953, Accuracy: 0.7620
Epoch 17, Loss: 0.5140, Accuracy: 0.7300
Epoch 18, Loss: 0.5242, Accuracy: 0.7340
Epoch 19, Loss: 0.5110, Accuracy: 0.7620
Epoch 20, Loss: 0.4728, Accuracy: 0.7760
Epoch 21, Loss: 0.4252, Accuracy: 0.7920
Epoch 22, Loss: 0.4417, Accuracy: 0.7920
Epoch 23, Loss: 0.3679, Accuracy: 0.8360
Epoch 24, Loss: 0.3635, Accuracy: 0.8260
Epoch 2